<a href="https://colab.research.google.com/github/SVNitinSV/Inventory_projection/blob/main/nlp_Tek_clover_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import Counter
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
path="/content/drive/MyDrive/ML Data - Outbound - updated.xlsx - Sheet1.csv"
df = pd.read_csv(path)
df.head(5)

In [ ]:
df.shape

In [ ]:
print("Original DataFrame:")
print(df.head())

# Keep only the specified columns
columns_to_keep = ['Item code', 'Item Description', 'Transaction Date', 'Qty']
df = df[columns_to_keep]

# Display the DataFrame after dropping columns
print("\nDataFrame after dropping unnecessary columns:")
print(df.head())

In [ ]:
df.head(5)

In [ ]:
nlp_df = df[['Item Description', 'Item code']]
nlp_df['Item Description'] = nlp_df['Item Description'].astype(str)

In [ ]:
nlp_df.head(5)

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
def preprocess_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special symbols
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back to string
    processed_text = ' '.join(tokens)
    return processed_text

In [ ]:
nlp_df['Item_Description_processed'] = nlp_df['Item Description'].apply(preprocess_text)

In [ ]:
nlp_df.sample(5)

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
text = ' '.join(nlp_df['Item_Description_processed'])

In [ ]:
wc = WordCloud(background_color='white',width=1920, height=1080).generate(text)
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
from textblob import TextBlob

In [ ]:
from gensim.models import Word2Vec

from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
text_list = nlp_df['Item_Description_processed'].tolist()
text_list = [nltk.word_tokenize(sentence.lower()) for sentence in text_list]


In [ ]:
print(text_list[0])

In [ ]:
!python -m pip install -U gensim

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/wiki-words/TensorFlow2/250/1")

In [ ]:
model = Word2Vec(text_list, vector_size=100, workers=1, seed=42)

In [ ]:
# Get word embeddings
word_embeddings = model.wv

In [ ]:
print(word_embeddings)

In [ ]:
# Iterate over the vocabulary and print embeddings for each word

model.wv.most_similar("spray")

In [ ]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

vectorized_docs = vectorize(text_list, model=model)
len(vectorized_docs), len(vectorized_docs[0])

In [ ]:
def mbkmeans_clusters(
	X,
    k,
    mb,
    print_silhouette_values,
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
          if not vectorized_docs or np.all(vectorized_docs == 0):
            raise ValueError("The vectorized_docs matrix is empty or contains all zeros.")
          cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
          silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
docs = nlp_df['Item_Description_processed'].values

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=300,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in text_list],
    "cluster": cluster_labels
})

In [ ]:
print("Most representative terms per cluster (based on centroids):")
for i in range(200):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=20)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")